In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import pandas as pd
from pandas import DataFrame

### Load and format the data

In [ ]:
parameterDF = pd.read_csv("ParameterData.csv", header=0, index_col=0, sep=',')
gravity, angle, stretch, length, mass, spring, success = parameterDF.values 

# Also group the successes and failures for plotting later
groups = parameterDF.T.groupby( 'Success' )
successGroup = groups.get_group(1)
failureGroup = groups.get_group(0)

### Prepare the KNN classifier and plots

In [ ]:
k = 1 # for nearest neighbors
h = .001  # step size in the mesh
cmap = ListedColormap( ['#F08080', '#98FB98'] )
params = ['Gravity', 'Initial Angle', 'Initial Stretch', 'Length', 'Mass', 'Spring']

In [ ]:
fig, axarr = plt.subplots( len(params), len(params), figsize=(15,15) )

for rowNum in range(len(params)):
    
    for colNum in range(len(params)):
        
        # Get training data for fitting KNN
        x1 = parameterDF.T[ params[colNum] ] 
        x2 = parameterDF.T[ params[rowNum] ]
        X = np.column_stack( (x1, x2) )
        classifier = neighbors.KNeighborsClassifier( k, weights='distance' ).fit( X, success )
    
        # Plot the decision boundary, assigning a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        yMin, yMax = x2.min(), x2.max()
        xMin, xMax = x1.min(), x1.max()
        xx, yy = np.meshgrid(np.arange(xMin-1, xMax+1, h),
                             np.arange(yMin-1, yMax+1, h))
        Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)       
        axarr[rowNum,colNum].pcolormesh(xx, yy, Z, cmap=cmap)
    
        # Plot the training data
        xSuccesses = successGroup[ params[colNum] ] 
        ySuccesses = successGroup[ params[rowNum] ]
        axarr[rowNum,colNum].scatter( xSuccesses, ySuccesses, marker='o', color='#006400', s=70 )
    
        xFailures = failureGroup[ params[colNum] ]
        yFailures = failureGroup[ params[rowNum] ]
        axarr[rowNum,colNum].scatter( xFailures, yFailures, marker='s', color='#FF0000', s=50 )

        axarr[rowNum,colNum].set_xlim( xMin-0.01, xMax+0.01 )
        axarr[rowNum,colNum].set_ylim( yMin-0.01, yMax+0.01 )

# Fine tune the plots        
count = 0 
for ax in axarr[-1, :]: # x-axis labels only for the bottom plots
    ax.set_xlabel( params[count] )
    count += 1 
    
for ax in axarr[0, :]: # x-axis tick labels on the top of the figure
    ax.xaxis.tick_top()
    
for rowNum in range( 1, len(params)-1 ): # hide x-axis tick labels for the middle rows
    plt.setp([ax.get_xticklabels() for ax in axarr[rowNum, :]], visible=False )

count = 0
for ax in axarr[:, 0]: # y-axis labels only for the left plots
    ax.set_ylabel( params[count] )
    count += 1   

for ax in axarr[:, -1]: # y-axis tick labels on the right of the figure
    ax.yaxis.tick_right()

for colNum in range( 1, len(params)-1 ): # hide y-axis tick labels for the middle columns
    plt.setp([ax.get_yticklabels() for ax in axarr[:, colNum]], visible=False )

fig.subplots_adjust(wspace=0.1, hspace=0.1)

plt.savefig('KNNPlot.png')